In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import shutil

# Directory containing MRI images
data_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer raw MRI data\\Brain_Cancer'
save_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer_processed_train_test'  # Directory to save resized images

# Create directories for train, val, and test
train_dir = os.path.join(save_dir, 'train')
val_dir = os.path.join(save_dir, 'val')
test_dir = os.path.join(save_dir, 'test')

for dir_path in [train_dir, val_dir, test_dir]:
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

# Collect all images and their paths with class subfolder information
all_images = []
for subfolder in os.listdir(data_dir):
    subfolder_path = os.path.join(data_dir, subfolder)
    
    if os.path.isdir(subfolder_path):
        for img_name in os.listdir(subfolder_path):
            img_path = os.path.join(subfolder_path, img_name)
            all_images.append((img_path, subfolder))  # Append the image path along with its class subfolder

# Split the data into 80% train, 10% validation, 10% test
train_images, temp_images = train_test_split(all_images, test_size=0.2, random_state=42)
val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

# Function to resize and save the images, maintaining class subfolders
def resize_and_save(image_list, base_save_dir):
    for img_path, subfolder in tqdm(image_list, desc=f"Processing {base_save_dir}"):
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        if image is None:
            continue

        # Resize to 256x256
        resized_image = cv2.resize(image, (256, 256))

        # Create the class subfolder in the target directory if it doesn't exist
        save_subfolder = os.path.join(base_save_dir, subfolder)
        if not os.path.exists(save_subfolder):
            os.makedirs(save_subfolder)

        # Save resized image in the correct class subfolder
        base_name = os.path.basename(img_path)
        save_path = os.path.join(save_subfolder, base_name)
        cv2.imwrite(save_path, resized_image)

# Resize and save for each dataset, maintaining the original class subfolder structure
resize_and_save(train_images, train_dir)
resize_and_save(val_images, val_dir)
resize_and_save(test_images, test_dir)

print(f"Data splitting and resizing complete. Processed images saved in {save_dir}.")


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import random

# Directory containing resized training images
train_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer_processed_train_test\\train'
augmented_dir = 'D:\\Brain Cancer\\Dataset\\Brain_Cancer_processed_train_test\\train_augmented'  # Directory to save augmented images

# Create the augmented directory if it doesn't exist
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Function for data augmentation
def augment_image(image):
    aug_images = []

    # Flipping horizontally
    flipped_image = cv2.flip(image, 1)
    aug_images.append(flipped_image)

    # Adding noise
    noise = np.random.normal(0, 0.0023**0.5, image.shape)
    noisy_image = image + noise
    aug_images.append(np.clip(noisy_image, 0, 1))  # Ensure pixel values are in [0, 1]

    # Adjusting contrast and brightness
    contrast_image = np.clip(0.79 * image, 0, 1)  # Adjust contrast
    brightness_image = np.clip(contrast_image + 0.23, 0, 1)  # Adjust brightness
    aug_images.append(brightness_image)

    # Random rotation
    rows, cols = image.shape[:2]
    random_angle = random.uniform(-15, 15)  # Choose a random angle between -15 and +15 degrees
    M = cv2.getRotationMatrix2D((cols / 2, rows / 2), random_angle, 1)
    rotated_image = cv2.warpAffine(image, M, (cols, rows))
    aug_images.append(rotated_image)

    return aug_images

# Loop through all subfolders and images in the train directory and apply augmentation
for subfolder in os.listdir(train_dir):
    subfolder_path = os.path.join(train_dir, subfolder)

    # Ensure the path is a directory
    if os.path.isdir(subfolder_path):
        # Create corresponding subfolder in the augmented directory
        save_subfolder = os.path.join(augmented_dir, subfolder)
        if not os.path.exists(save_subfolder):
            os.makedirs(save_subfolder)

        # Loop through all images in the current subfolder
        for img_name in tqdm(os.listdir(subfolder_path), desc=f"Augmenting {subfolder}"):
            img_path = os.path.join(subfolder_path, img_name)
            image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
            if image is None:
                continue

            # Apply augmentation
            augmented_images = augment_image(image)

            # Save the original image (resized) and augmented images in the corresponding class folder
            base_name, ext = os.path.splitext(img_name)
            cv2.imwrite(os.path.join(save_subfolder, f"{base_name}_resized{ext}"), image)  # Save original resized image

            for i, aug_image in enumerate(augmented_images):
                save_path = os.path.join(save_subfolder, f"{base_name}_aug_{i}{ext}")
                cv2.imwrite(save_path, aug_image * 255)  # Save augmented image

print("Augmentation complete for training data.")